# Summaries Visualization

In [1]:
import json
import pandas as pd

RAWDATAFILES = {
    "train": "training_complete.jsonl",
    "val": "validation_complete.jsonl",
    "test": "testing_with_paper_release.jsonl"
}

In [82]:
def import_cluster(data_split, features = 'Ent_Rel',model='kmeans-k3',ver=1):
    filepath = f"clustering/{features}/{model}-{data_split}-v{ver}.csv"
    print(f"importing clusters from {filepath}")
    return pd.read_csv(filepath).drop_duplicates().drop(columns='sum_id')

def import_fullpaper(data_split, raw=True):
    if raw: 
        filepath = f"dataset_MuP/{RAWDATAFILES[data_split]}" 
    else: 
        filepath = f"{main_path}/dataset_MuP/{data_split}_iden.jsonl"
    print(f"importing full paper from {filepath}")
    with open(filepath, 'r') as json_file:
        json_list = list(json_file)
    result = []
    for i, json_str in enumerate(json_list):
        data = json.loads(json_str)
        result.append({
            'paper_id': data['paper_id'], 
            'heading': 'ABSTRACT', 
            'text': data['paper']['abstractText'],
        })
    
        sections = [{'paper_id': data['paper_id'], **sec} for sec in data['paper']['sections']]
        result += sections
    return  pd.DataFrame(result).drop_duplicates()

def import_graph(data_split):
    filepath = f"PL-Marker/_scire_models/MuP/summary/{data_split}_re.json"
    print(f"importing graph data from {filepath}")
    with open(filepath, 'r') as json_file:
        json_list = list(json_file)
    return pd.DataFrame([json.loads(json_str) for json_str in json_list]).drop(columns=['ner', 'relations'])

def import_gen_sum(data_split):
    model = "bart-large-cnn_abstract+introduction-1+conclusion_text"
    filepath = f"finetuning/generated_summary/{data_split}/{model}.csv"
    return pd.read_csv(filepath)
    
        
    

In [85]:
def sent2sum(sent_list):
    return ' '.join([' '.join(sent) for sent in sent_list])
    
def prepare_df(data_split):
    cluster_df = import_cluster(data_split)
    fullpaper_df = import_fullpaper(data_split)
    graph_df = import_graph(data_split)
    graph_df['summary'] = graph_df['sentences'].apply(sent2sum)
    graph_df.rename(columns={'doc_key': 'paper_id'}, inplace=True)
    merge_sum_df = cluster_df.merge(graph_df, on=['paper_id', 'summary'])
    return merge_sum_df, fullpaper_df

## Train set

In [96]:
cluster_df = import_cluster("train")
fullpaper_df = import_fullpaper("train")
graph_df = import_graph("train")

importing clusters from clustering/Ent_Rel/kmeans-k3-train-v1.csv
importing full paper from dataset_MuP/training_complete.jsonl
importing graph data from PL-Marker/_scire_models/MuP/summary/train_re.json


In [97]:
def sent2sum(sent_list):
    return ' '.join([' '.join(sent) for sent in sent_list])

In [98]:
graph_df['summary'] = graph_df['sentences'].apply(sent2sum)
graph_df.rename(columns={'doc_key': 'paper_id'}, inplace=True)
graph_df

,paper_id,sentences,predicted_ner,predicted_re,summary
0,SP:4d08cdb2de2044bcb574a425b42963b83fbebfbc,"[[This, paper, investigates, kernel, ridge, -,...","[[[3, 7, Method], [15, 16, OtherScientificTerm...","[[0, []], [1, [[[19, 20], [23, 24], 'USED-FOR'...",This paper investigates kernel ridge - less re...
1,SP:b80bc890180934092cde037b49d94d6e4e06fad9,"[[This, paper, presents, a, novel, way, of, ma...","[[[11, 13, Method], [16, 17, OtherScientificTe...","[[0, [[[11, 13], [16, 17], 'USED-FOR'], [[16, ...",This paper presents a novel way of making full...
2,SP:09f2fe6a482bbd6f9bd2c62aa841f995171ba939,"[[This, paper, proposes, a, new, framework, th...","[[[5, 5, Generic], [9, 12, Method], [16, 17, O...","[[0, [[[9, 12], [16, 17], 'USED-FOR'], [[5, 5]...",This paper proposes a new framework that compu...
3,SP:a1e2218e6943bf138aeb359e23628676b396ed66,"[[This, work, proposes, a, deep, reinforcement...","[[[4, 10, Method], [13, 15, Task], [18, 20, Ta...","[[0, [[[4, 10], [13, 15], 'USED-FOR'], [[13, 1...",This work proposes a deep reinforcement learni...
4,SP:43e525fb3fa611df7fd44bd3bc9843e57b154c66,"[[This, paper, proposes, 3, deep, generative, ...","[[[4, 6, Method], [9, 9, Method], [13, 14, Oth...","[[0, [[[4, 6], [22, 26], 'USED-FOR'], [[13, 14...",This paper proposes 3 deep generative models b...
...,...,...,...,...,...
18929,SP:0d872fb4321f3a4a3fc61cf4d33b0c7e33f2d695,"[[This, paper, presents, a, RNN, -, RL, based,...","[[[4, 8, Method], [11, 13, Task]], [[16, 16, G...","[[0, [[[4, 8], [11, 13], 'USED-FOR']]], [1, []...",This paper presents a RNN - RL based method fo...
18930,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,"[[This, paper, proposes, a, new, pointwise, co...","[[[5, 7, Method]], [[27, 27, Generic], [31, 31...","[[0, []], [1, [[[33, 33], [37, 37], 'USED-FOR'...",This paper proposes a new pointwise convolutio...
18931,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,"[[This, paper, presents, a, new, pointwise, co...","[[[5, 10, Method], [14, 14, Generic], [17, 17,...","[[0, [[[17, 17], [19, 19], 'CONJUNCTION'], [[1...",This paper presents a new pointwise convolutio...
18932,SP:63ad3be1dae7ede5c02a847304072c1cbc91b1cb,"[[This, paper, proposes, to, model, various, u...","[[[6, 7, Metric], [9, 14, Method], [16, 18, Me...","[[0, [[[6, 7], [9, 14], 'PART-OF'], [[16, 18],...",This paper proposes to model various uncertain...


In [99]:
merge_sum_df = cluster_df.merge(graph_df, on=['paper_id', 'summary'])
merge_sum_df

,paper_id,summary,cluster,sentences,predicted_ner,predicted_re
0,SP:4d08cdb2de2044bcb574a425b42963b83fbebfbc,This paper investigates kernel ridge - less re...,2,"[[This, paper, investigates, kernel, ridge, -,...","[[[3, 7, Method], [15, 16, OtherScientificTerm...","[[0, []], [1, [[[19, 20], [23, 24], 'USED-FOR'..."
1,SP:b80bc890180934092cde037b49d94d6e4e06fad9,This paper presents a novel way of making full...,2,"[[This, paper, presents, a, novel, way, of, ma...","[[[11, 13, Method], [16, 17, OtherScientificTe...","[[0, [[[11, 13], [16, 17], 'USED-FOR'], [[16, ..."
2,SP:09f2fe6a482bbd6f9bd2c62aa841f995171ba939,This paper proposes a new framework that compu...,2,"[[This, paper, proposes, a, new, framework, th...","[[[5, 5, Generic], [9, 12, Method], [16, 17, O...","[[0, [[[9, 12], [16, 17], 'USED-FOR'], [[5, 5]..."
3,SP:a1e2218e6943bf138aeb359e23628676b396ed66,This work proposes a deep reinforcement learni...,2,"[[This, work, proposes, a, deep, reinforcement...","[[[4, 10, Method], [13, 15, Task], [18, 20, Ta...","[[0, [[[4, 10], [13, 15], 'USED-FOR'], [[13, 1..."
4,SP:43e525fb3fa611df7fd44bd3bc9843e57b154c66,This paper proposes 3 deep generative models b...,1,"[[This, paper, proposes, 3, deep, generative, ...","[[[4, 6, Method], [9, 9, Method], [13, 14, Oth...","[[0, [[[4, 6], [22, 26], 'USED-FOR'], [[13, 14..."
...,...,...,...,...,...,...
18929,SP:0d872fb4321f3a4a3fc61cf4d33b0c7e33f2d695,This paper presents a RNN - RL based method fo...,2,"[[This, paper, presents, a, RNN, -, RL, based,...","[[[4, 8, Method], [11, 13, Task]], [[16, 16, G...","[[0, [[[4, 8], [11, 13], 'USED-FOR']]], [1, []..."
18930,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,This paper proposes a new pointwise convolutio...,2,"[[This, paper, proposes, a, new, pointwise, co...","[[[5, 7, Method]], [[27, 27, Generic], [31, 31...","[[0, []], [1, [[[33, 33], [37, 37], 'USED-FOR'..."
18931,SP:4706017e6f8b958c7d0825fed98b285ea2994b59,This paper presents a new pointwise convolutio...,0,"[[This, paper, presents, a, new, pointwise, co...","[[[5, 10, Method], [14, 14, Generic], [17, 17,...","[[0, [[[17, 17], [19, 19], 'CONJUNCTION'], [[1..."
18932,SP:63ad3be1dae7ede5c02a847304072c1cbc91b1cb,This paper proposes to model various uncertain...,1,"[[This, paper, proposes, to, model, various, u...","[[[6, 7, Metric], [9, 14, Method], [16, 18, Me...","[[0, [[[6, 7], [9, 14], 'PART-OF'], [[16, 18],..."


In [100]:
paper_diff_list = []
for paper_id in set(merge_sum_df.paper_id):
    df = merge_sum_df[merge_sum_df.paper_id==paper_id]
    if len(set(df.cluster))==3: paper_diff_list.append(paper_id)


In [107]:
n = 5
paper_id = paper_diff_list[n]
paper_id = "SP:9e9fdc790ae44306844c7ae6c82aaf2dc8e69e0e"
df = merge_sum_df[merge_sum_df.paper_id==paper_id]
for idx, row in df.iterrows():
    print(f"Cluster {row.cluster}: {row.summary}")

Cluster 2: The paper analyzes the relationship between relative quantization errors and fixed - point formats for zero - centered normal distributions and finds a linear model which fits the best exponent length of the fixed - point data type given a standard deviation . These insights are then unified with parameterized clipping activation ( PACT ) to normalize incoming floating point data into the desired fixed point range . To handle the network with sole 8 - bit multiplications a forward pass in floating precision is used to compute batch norm statistics for the main 8 - bit forward / backward pass . Additional adjustments are made between successive layers and residual layers which rely on reusing some statistics of the previous layer .
Cluster 1: The authors propose a new quantization flow to train DNNs using only 8 - bit fixed - point multiplications . They show that 8 - bit fixed point can represent different exponent ranges based on fractional length , thus choosing the right 

In [108]:
fullpaper_df[fullpaper_df.paper_id==paper_id]

,paper_id,heading,text
195895,SP:9e9fdc790ae44306844c7ae6c82aaf2dc8e69e0e,ABSTRACT,Neural network quantization is a promising com...
195896,SP:9e9fdc790ae44306844c7ae6c82aaf2dc8e69e0e,1 INTRODUCTION,Real-time inference on resource-constrained an...
195897,SP:9e9fdc790ae44306844c7ae6c82aaf2dc8e69e0e,2 RELATED WORK,Quantization is one of the most widely-used te...
195898,SP:9e9fdc790ae44306844c7ae6c82aaf2dc8e69e0e,3 ANALYSIS OF FIXED-POINT REPRESENTATION,"In this section, we first introduce the fixed-..."
195899,SP:9e9fdc790ae44306844c7ae6c82aaf2dc8e69e0e,3.1 ADVANTAGES OF FIXED-POINT ARITHMETIC,Fixed-point number is characterized by its for...
195900,SP:9e9fdc790ae44306844c7ae6c82aaf2dc8e69e0e,3.2 STATISTICAL ANALYSIS FOR FIXED-POINT FORMAT,"For a predefined bit-width, integer, which is ..."
195901,SP:9e9fdc790ae44306844c7ae6c82aaf2dc8e69e0e,3.3 CHOOSING OPTIMAL FIXED-POINT FORMAT,"With the above observations, we are interested..."
195902,SP:9e9fdc790ae44306844c7ae6c82aaf2dc8e69e0e,4 METHODS,"In this section, we discuss our proposed train..."
195903,SP:9e9fdc790ae44306844c7ae6c82aaf2dc8e69e0e,4.1 UNIFYING PACT AND FIXED-POINT QUANTIZATION,To quantize a positive value x with unsigned f...
195904,SP:9e9fdc790ae44306844c7ae6c82aaf2dc8e69e0e,4.2 UPDATING BN AND FRACTIONAL LENGTH,Double Forward for BN Fusion. To quantize the ...


## Validation set

In [87]:
merge_sum_df, fullpaper_df = prepare_df('val')
merge_sum_df

importing clusters from clustering/Ent_Rel/kmeans-k3-val-v1.csv
importing full paper from dataset_MuP/validation_complete.jsonl
importing graph data from PL-Marker/_scire_models/MuP/summary/val_re.json


,paper_id,summary,cluster,sentences,predicted_ner,predicted_re
0,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,This paper studies FL under local differential...,2,"[[This, paper, studies, FL, under, local, diff...","[[[3, 3, Task], [5, 8, OtherScientificTerm]], ...","[[0, [[[5, 8], [3, 3], 'FEATURE-OF']]], [1, [[..."
1,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,This paper studies a low communication algorit...,1,"[[This, paper, studies, a, low, communication,...","[[[4, 6, Method], [8, 10, Task], [13, 15, Mate...","[[0, [[[4, 6], [8, 10], 'USED-FOR'], [[13, 15]..."
2,SP:7f4b788b00a2a10bcd60351c3e04c8f597101e96,The paper proposed a differentially private tr...,1,"[[The, paper, proposed, a, differentially, pri...","[[[4, 7, Method], [9, 10, Method]], [[17, 18, ...","[[0, [[[4, 7], [9, 10], 'USED-FOR']]], [1, []]..."
3,SP:632666b52c7c551d67fbbe70c06ed589c3a5e187,This submission works on the neural machine tr...,1,"[[This, submission, works, on, the, neural, ma...","[[[5, 8, Task]], [[18, 19, OtherScientificTerm...","[[0, []], [1, [[[18, 19], [21, 22], 'CONJUNCTI..."
4,SP:632666b52c7c551d67fbbe70c06ed589c3a5e187,This paper proposes a method to introduce * * ...,2,"[[This, paper, proposes, a, method, to, introd...","[[[4, 4, Generic], [9, 10, OtherScientificTerm...","[[0, [[[14, 18], [25, 30], 'USED-FOR'], [[4, 4..."
...,...,...,...,...,...,...
3599,SP:18aaba3423e81e9437b509d1a5e24836ef5635f6,This paper defines a set of learnable basis fu...,2,"[[This, paper, defines, a, set, of, learnable,...","[[[11, 13, Method], [16, 16, Generic]], [[26, ...","[[0, [[[11, 13], [16, 16], 'USED-FOR']]], [1, ..."
3600,SP:18aaba3423e81e9437b509d1a5e24836ef5635f6,A typical Wavelet Transform is built through t...,2,"[[A, typical, Wavelet, Transform, is, built, t...","[[[2, 3, Method], [8, 14, OtherScientificTerm]...","[[0, [[[22, 23], [26, 27], 'FEATURE-OF'], [[8,..."
3601,SP:3058e6bc5e8c62af325c214c9e1436d6cdf09204,This paper builds a new graph convolutional ne...,1,"[[This, paper, builds, a, new, graph, convolut...","[[[5, 10, Method], [13, 18, Method], [21, 25, ...","[[0, [[[13, 18], [5, 10], 'USED-FOR']]], [1, [..."
3602,SP:3058e6bc5e8c62af325c214c9e1436d6cdf09204,The authors propose using non - Euclidean spac...,2,"[[The, authors, propose, using, non, -, Euclid...","[[[4, 7, OtherScientificTerm], [9, 9, Method]]...","[[0, [[[4, 7], [9, 9], 'USED-FOR']]], [2, [[[3..."


In [89]:
paper_diff_list = []
for paper_id in set(merge_sum_df.paper_id):
    df = merge_sum_df[merge_sum_df.paper_id==paper_id]
    if len(set(df.cluster))==3: paper_diff_list.append(paper_id)

In [90]:
n = 2
paper_id = paper_diff_list[2]
df = merge_sum_df[merge_sum_df.paper_id==paper_id]
for idx, row in df.iterrows():
    print(f"Cluster {row.cluster}: {row.summary}")

Cluster 2: This paper adopts the framework of unsupervised skill learning . To solve the problem that the discriminator will have low confidence in the unseen data thus providing a low intrinsic reward , the paper derives an information gain auxiliary objective that involves training an ensemble of discriminators and rewarding the policy for their disagreement . The paper conducts extensive experiments on tabular grid world and 57 games of the Atari Suite .
Cluster 1: The paper proposes a novel unsupervised skill discovery algorithm . Beginning with the family of such methods ( DIAYN , VIC , etc ) that learn a discriminator to distinguish skills given some observations of the trajectory and a policy that executes a skill conditioned on the ( discrete ) skill random variable Z , the premise of the paper is that such methods on their own will fail when new states are encountered during the skill learning process as the discriminator would not have had sufficient data to learn to distingu

In [93]:
gen_sum_df = import_gen_sum('val')
for sum in gen_sum_df[gen_sum_df.paper_id==paper_id]['summary']: print(sum)

This paper studies the problem of unsupervised skill learning. The authors propose an exploration bonus that penalizes the discriminator for not having seen enough training data to produce accurate and confident skill classifications, leading to a low intrinsic reward for the agent and effective penalization of the sort of exploration needed to actually maximize the objective. To solve this problem, the authors derive an information gain auxiliary objective that involves training an ensemble of discriminators and rewarding the policy for their disagreement. They demonstrate empirically that DISDAIN improves skill learning both in a tabular grid world (Four Rooms) and the 57 games of the Atari Suite.
